# PersonAttributes Identification

In [0]:
!nvidia-smi

## Import the dataset and setup train and valid generators with augmentations

In [0]:
# mount gdrive and unzip data
from google.colab import drive
drive.mount('/content/gdrive')
!unzip -q "/content/gdrive/My Drive/hvc_data.zip"
# look for `hvc_annotations.csv` file and `resized` dir
%ls 

In [0]:
# Install fastparquet and Pillow files. These would help to store train, valid dataframes and use Auto augment policies. Storage of train , valid dataframes after train,valid split is important to prevent data leak when the weights were loaded and trained again.

!pip install fastparquet
!pip install Pillow -U

In [0]:
# Load all libraries and use tensorflow 2.0 in colab environment


%tensorflow_version 2.x

import cv2
import json

import numpy as np
import pandas as pd

from functools import partial
from pathlib import Path 
from tqdm import tqdm

from google.colab.patches import cv2_imshow

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten, GlobalAveragePooling2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [0]:
# load annotations
df = pd.read_csv("hvc_annotations.csv")
del df["filename"] # remove unwanted column
df.head()

In [0]:
# one hot encoding of labels

one_hot_df = pd.concat([
    df[["image_path"]],
    pd.get_dummies(df.gender, prefix="gender"),
    pd.get_dummies(df.imagequality, prefix="imagequality"),
    pd.get_dummies(df.age, prefix="age"),
    pd.get_dummies(df.weight, prefix="weight"),
    pd.get_dummies(df.carryingbag, prefix="carryingbag"),
    pd.get_dummies(df.footwear, prefix="footwear"),
    pd.get_dummies(df.emotion, prefix="emotion"),
    pd.get_dummies(df.bodypose, prefix="bodypose"),
], axis = 1)

one_hot_df.head().T

In [0]:
# Use auto-augment policies from 

# Ref : https://towardsdatascience.com/how-to-improve-your-image-classifier-with-googles-autoaugment-77643f0be0c9
# Ref : https://github.com/DeepVoltaire/AutoAugment
# Ref : https://arxiv.org/abs/1805.09501v1

from PIL import Image, ImageEnhance, ImageOps
import numpy as np
import random


class ImageNetPolicy(object):
    """ Randomly choose one of the best 24 Sub-policies on ImageNet.
        Example:
        >>> policy = ImageNetPolicy()
        >>> transformed = policy(image)
        Example as a PyTorch Transform:
        >>> transform=transforms.Compose([
        >>>     transforms.Resize(256),
        >>>     ImageNetPolicy(),
        >>>     transforms.ToTensor()])
    """
    def __init__(self, fillcolor=(128, 128, 128)):
        self.policies = [
            SubPolicy(0.4, "posterize", 8, 0.6, "rotate", 9, fillcolor),
            SubPolicy(0.6, "solarize", 5, 0.6, "autocontrast", 5, fillcolor),
            SubPolicy(0.8, "equalize", 8, 0.6, "equalize", 3, fillcolor),
            SubPolicy(0.6, "posterize", 7, 0.6, "posterize", 6, fillcolor),
            SubPolicy(0.4, "equalize", 7, 0.2, "solarize", 4, fillcolor),

            SubPolicy(0.4, "equalize", 4, 0.8, "rotate", 8, fillcolor),
            SubPolicy(0.6, "solarize", 3, 0.6, "equalize", 7, fillcolor),
            SubPolicy(0.8, "posterize", 5, 1.0, "equalize", 2, fillcolor),
            SubPolicy(0.2, "rotate", 3, 0.6, "solarize", 8, fillcolor),
            SubPolicy(0.6, "equalize", 8, 0.4, "posterize", 6, fillcolor),

            SubPolicy(0.8, "rotate", 8, 0.4, "color", 0, fillcolor),
            SubPolicy(0.4, "rotate", 9, 0.6, "equalize", 2, fillcolor),
            SubPolicy(0.0, "equalize", 7, 0.8, "equalize", 8, fillcolor),
            SubPolicy(0.6, "invert", 4, 1.0, "equalize", 8, fillcolor),
            SubPolicy(0.6, "color", 4, 1.0, "contrast", 8, fillcolor),

            SubPolicy(0.8, "rotate", 8, 1.0, "color", 2, fillcolor),
            SubPolicy(0.8, "color", 8, 0.8, "solarize", 7, fillcolor),
            SubPolicy(0.4, "sharpness", 7, 0.6, "invert", 8, fillcolor),
            SubPolicy(0.6, "shearX", 5, 1.0, "equalize", 9, fillcolor),
            SubPolicy(0.4, "color", 0, 0.6, "equalize", 3, fillcolor),

            SubPolicy(0.4, "equalize", 7, 0.2, "solarize", 4, fillcolor),
            SubPolicy(0.6, "solarize", 5, 0.6, "autocontrast", 5, fillcolor),
            SubPolicy(0.6, "invert", 4, 1.0, "equalize", 8, fillcolor),
            SubPolicy(0.6, "color", 4, 1.0, "contrast", 8, fillcolor),
            SubPolicy(0.8, "equalize", 8, 0.6, "equalize", 3, fillcolor)
        ]


    def __call__(self, img):
        policy_idx = random.randint(0, len(self.policies) - 1)
        return self.policies[policy_idx](img)

    def __repr__(self):
        return "AutoAugment ImageNet Policy"


class CIFAR10Policy(object):
    """ Randomly choose one of the best 25 Sub-policies on CIFAR10.
        Example:
        >>> policy = CIFAR10Policy()
        >>> transformed = policy(image)
        Example as a PyTorch Transform:
        >>> transform=transforms.Compose([
        >>>     transforms.Resize(256),
        >>>     CIFAR10Policy(),
        >>>     transforms.ToTensor()])
    """
    def __init__(self, fillcolor=(128, 128, 128)):
        self.policies = [
            SubPolicy(0.1, "invert", 7, 0.2, "contrast", 6, fillcolor),
            SubPolicy(0.7, "rotate", 2, 0.3, "translateX", 9, fillcolor),
            SubPolicy(0.8, "sharpness", 1, 0.9, "sharpness", 3, fillcolor),
            SubPolicy(0.5, "shearY", 8, 0.7, "translateY", 9, fillcolor),
            SubPolicy(0.5, "autocontrast", 8, 0.9, "equalize", 2, fillcolor),

            SubPolicy(0.2, "shearY", 7, 0.3, "posterize", 7, fillcolor),
            SubPolicy(0.4, "color", 3, 0.6, "brightness", 7, fillcolor),
            SubPolicy(0.3, "sharpness", 9, 0.7, "brightness", 9, fillcolor),
            SubPolicy(0.6, "equalize", 5, 0.5, "equalize", 1, fillcolor),
            SubPolicy(0.6, "contrast", 7, 0.6, "sharpness", 5, fillcolor),

            SubPolicy(0.7, "color", 7, 0.5, "translateX", 8, fillcolor),
            SubPolicy(0.3, "equalize", 7, 0.4, "autocontrast", 8, fillcolor),
            SubPolicy(0.4, "translateY", 3, 0.2, "sharpness", 6, fillcolor),
            SubPolicy(0.9, "brightness", 6, 0.2, "color", 8, fillcolor),
            SubPolicy(0.5, "solarize", 2, 0.0, "invert", 3, fillcolor),

            SubPolicy(0.2, "equalize", 0, 0.6, "autocontrast", 0, fillcolor),
            SubPolicy(0.2, "equalize", 8, 0.6, "equalize", 4, fillcolor),
            SubPolicy(0.9, "color", 9, 0.6, "equalize", 6, fillcolor),
            SubPolicy(0.8, "autocontrast", 4, 0.2, "solarize", 8, fillcolor),
            SubPolicy(0.1, "brightness", 3, 0.7, "color", 0, fillcolor),

            SubPolicy(0.4, "solarize", 5, 0.9, "autocontrast", 3, fillcolor),
            SubPolicy(0.9, "translateY", 9, 0.7, "translateY", 9, fillcolor),
            SubPolicy(0.9, "autocontrast", 2, 0.8, "solarize", 3, fillcolor),
            SubPolicy(0.8, "equalize", 8, 0.1, "invert", 3, fillcolor),
            SubPolicy(0.7, "translateY", 9, 0.9, "autocontrast", 1, fillcolor)
        ]


    def __call__(self, img):
        policy_idx = random.randint(0, len(self.policies) - 1)
        return self.policies[policy_idx](img)

    def __repr__(self):
        return "AutoAugment CIFAR10 Policy"


class SVHNPolicy(object):
    """ Randomly choose one of the best 25 Sub-policies on SVHN.
        Example:
        >>> policy = SVHNPolicy()
        >>> transformed = policy(image)
        Example as a PyTorch Transform:
        >>> transform=transforms.Compose([
        >>>     transforms.Resize(256),
        >>>     SVHNPolicy(),
        >>>     transforms.ToTensor()])
    """
    def __init__(self, fillcolor=(128, 128, 128)):
        self.policies = [
            SubPolicy(0.9, "shearX", 4, 0.2, "invert", 3, fillcolor),
            SubPolicy(0.9, "shearY", 8, 0.7, "invert", 5, fillcolor),
            SubPolicy(0.6, "equalize", 5, 0.6, "solarize", 6, fillcolor),
            SubPolicy(0.9, "invert", 3, 0.6, "equalize", 3, fillcolor),
            SubPolicy(0.6, "equalize", 1, 0.9, "rotate", 3, fillcolor),

            SubPolicy(0.9, "shearX", 4, 0.8, "autocontrast", 3, fillcolor),
            SubPolicy(0.9, "shearY", 8, 0.4, "invert", 5, fillcolor),
            SubPolicy(0.9, "shearY", 5, 0.2, "solarize", 6, fillcolor),
            SubPolicy(0.9, "invert", 6, 0.8, "autocontrast", 1, fillcolor),
            SubPolicy(0.6, "equalize", 3, 0.9, "rotate", 3, fillcolor),

            SubPolicy(0.9, "shearX", 4, 0.3, "solarize", 3, fillcolor),
            SubPolicy(0.8, "shearY", 8, 0.7, "invert", 4, fillcolor),
            SubPolicy(0.9, "equalize", 5, 0.6, "translateY", 6, fillcolor),
            SubPolicy(0.9, "invert", 4, 0.6, "equalize", 7, fillcolor),
            SubPolicy(0.3, "contrast", 3, 0.8, "rotate", 4, fillcolor),

            SubPolicy(0.8, "invert", 5, 0.0, "translateY", 2, fillcolor),
            SubPolicy(0.7, "shearY", 6, 0.4, "solarize", 8, fillcolor),
            SubPolicy(0.6, "invert", 4, 0.8, "rotate", 4, fillcolor),
            SubPolicy(0.3, "shearY", 7, 0.9, "translateX", 3, fillcolor),
            SubPolicy(0.1, "shearX", 6, 0.6, "invert", 5, fillcolor),

            SubPolicy(0.7, "solarize", 2, 0.6, "translateY", 7, fillcolor),
            SubPolicy(0.8, "shearY", 4, 0.8, "invert", 8, fillcolor),
            SubPolicy(0.7, "shearX", 9, 0.8, "translateY", 3, fillcolor),
            SubPolicy(0.8, "shearY", 5, 0.7, "autocontrast", 3, fillcolor),
            SubPolicy(0.7, "shearX", 2, 0.1, "invert", 5, fillcolor)
        ]


    def __call__(self, img):
        policy_idx = random.randint(0, len(self.policies) - 1)
        return self.policies[policy_idx](img)

    def __repr__(self):
        return "AutoAugment SVHN Policy"


class SubPolicy(object):
    def __init__(self, p1, operation1, magnitude_idx1, p2, operation2, magnitude_idx2, fillcolor=(128, 128, 128)):
        ranges = {
            "shearX": np.linspace(0, 0.3, 10),
            "shearY": np.linspace(0, 0.3, 10),
            "translateX": np.linspace(0, 150 / 331, 10),
            "translateY": np.linspace(0, 150 / 331, 10),
            "rotate": np.linspace(0, 30, 10),
            "color": np.linspace(0.0, 0.9, 10),
            "posterize": np.round(np.linspace(8, 4, 10), 0).astype(np.int),
            "solarize": np.linspace(256, 0, 10),
            "contrast": np.linspace(0.0, 0.9, 10),
            "sharpness": np.linspace(0.0, 0.9, 10),
            "brightness": np.linspace(0.0, 0.9, 10),
            "autocontrast": [0] * 10,
            "equalize": [0] * 10,
            "invert": [0] * 10
        }

        # from https://stackoverflow.com/questions/5252170/specify-image-filling-color-when-rotating-in-python-with-pil-and-setting-expand
        def rotate_with_fill(img, magnitude):
            rot = img.convert("RGBA").rotate(magnitude)
            return Image.composite(rot, Image.new("RGBA", rot.size, (128,) * 4), rot).convert(img.mode)

        func = {
            "shearX": lambda img, magnitude: img.transform(
                img.size, Image.AFFINE, (1, magnitude * random.choice([-1, 1]), 0, 0, 1, 0),
                Image.BICUBIC, fillcolor=fillcolor),
            "shearY": lambda img, magnitude: img.transform(
                img.size, Image.AFFINE, (1, 0, 0, magnitude * random.choice([-1, 1]), 1, 0),
                Image.BICUBIC, fillcolor=fillcolor),
            "translateX": lambda img, magnitude: img.transform(
                img.size, Image.AFFINE, (1, 0, magnitude * img.size[0] * random.choice([-1, 1]), 0, 1, 0),
                fillcolor=fillcolor),
            "translateY": lambda img, magnitude: img.transform(
                img.size, Image.AFFINE, (1, 0, 0, 0, 1, magnitude * img.size[1] * random.choice([-1, 1])),
                fillcolor=fillcolor),
            "rotate": lambda img, magnitude: rotate_with_fill(img, magnitude),
            "color": lambda img, magnitude: ImageEnhance.Color(img).enhance(1 + magnitude * random.choice([-1, 1])),
            "posterize": lambda img, magnitude: ImageOps.posterize(img, magnitude),
            "solarize": lambda img, magnitude: ImageOps.solarize(img, magnitude),
            "contrast": lambda img, magnitude: ImageEnhance.Contrast(img).enhance(
                1 + magnitude * random.choice([-1, 1])),
            "sharpness": lambda img, magnitude: ImageEnhance.Sharpness(img).enhance(
                1 + magnitude * random.choice([-1, 1])),
            "brightness": lambda img, magnitude: ImageEnhance.Brightness(img).enhance(
                1 + magnitude * random.choice([-1, 1])),
            "autocontrast": lambda img, magnitude: ImageOps.autocontrast(img),
            "equalize": lambda img, magnitude: ImageOps.equalize(img),
            "invert": lambda img, magnitude: ImageOps.invert(img)
        }

        self.p1 = p1
        self.operation1 = func[operation1]
        self.magnitude1 = ranges[operation1][magnitude_idx1]
        self.p2 = p2
        self.operation2 = func[operation2]
        self.magnitude2 = ranges[operation2][magnitude_idx2]


    def __call__(self, img):
        if random.random() < self.p1: img = self.operation1(img, self.magnitude1)
        if random.random() < self.p2: img = self.operation2(img, self.magnitude2)
        return img

In [0]:
# Import necessary libraries

%reload_ext autoreload
%autoreload 2

from PIL import Image, ImageEnhance, ImageOps
import numpy as np
import matplotlib.pyplot as plt
import math
import random
import pdb
#from autoaugment import ImageNetPolicy, CIFAR10Policy, SVHNPolicy, SubPolicy


In [0]:
# Create a generator using tf.keras.utils.Sequence. The generator also need to apply necessary augmentations and AutoAugment policies.

from  skimage import transform
from PIL import Image, ImageEnhance, ImageOps


import tensorflow.keras as keras
import numpy as np

# Label columns per attribute
_gender_cols_ = [col for col in one_hot_df.columns if col.startswith("gender")]
_imagequality_cols_ = [col for col in one_hot_df.columns if col.startswith("imagequality")]
_age_cols_ = [col for col in one_hot_df.columns if col.startswith("age")]
_weight_cols_ = [col for col in one_hot_df.columns if col.startswith("weight")]
_carryingbag_cols_ = [col for col in one_hot_df.columns if col.startswith("carryingbag")]
_footwear_cols_ = [col for col in one_hot_df.columns if col.startswith("footwear")]
_emotion_cols_ = [col for col in one_hot_df.columns if col.startswith("emotion")]
_bodypose_cols_ = [col for col in one_hot_df.columns if col.startswith("bodypose")]

class PersonDataGenerator(keras.utils.Sequence):
    """Ground truth data generator"""

    
    def __init__(self, df, augmentations, policy = None, new_shape = (224,224) , batch_size=32, shuffle=True):
        self.df = df
        self.batch_size=batch_size
        self.shuffle = shuffle
        self.augment = augmentations
        self.policy = policy
        self.on_epoch_end()

    def __len__(self):
        return int(np.floor(self.df.shape[0] / self.batch_size))

    def __getitem__(self, index):
        """fetch batched images and targets"""
        policy = ImageNetPolicy()
        batch_slice = slice(index * self.batch_size, (index + 1) * self.batch_size)
        items = self.df.iloc[batch_slice]
        #image = np.stack([self.augment(policy(cv2.resize(cv2.imread(item["image_path"]), new_shape))) for _, item in items.iterrows()])
        if not self.policy:
          image = np.stack([self.augment(cv2.resize(np.array(Image.open(item["image_path"])), new_shape)) for _, item in items.iterrows()])
        else:
          image = np.stack([self.augment(cv2.resize(np.array(self.policy(Image.open(item["image_path"]))), new_shape)) for _, item in items.iterrows()])

        target = {
            "gender_output": items[_gender_cols_].values,
            "image_quality_output": items[_imagequality_cols_].values,
            "age_output": items[_age_cols_].values,
            "weight_output": items[_weight_cols_].values,
            "bag_output": items[_carryingbag_cols_].values,
            "pose_output": items[_bodypose_cols_].values,
            "footwear_output": items[_footwear_cols_].values,
            "emotion_output": items[_emotion_cols_].values,
        }
        return image, target

    def on_epoch_end(self):
        """Updates indexes after each epoch"""
        if self.shuffle == True:
            self.df = self.df.sample(frac=1).reset_index(drop=True)


In [0]:
# Perform train and test split

from sklearn.model_selection import train_test_split
train_df_sk, val_df_sk = train_test_split(one_hot_df, test_size=0.15)
train_df_sk.shape, val_df_sk.shape

In [0]:
# Store the test , train dataframes as parquet files. This is to prevent data leak. Run this the first time and comment the code so that this is not executed for subsequent runs in Colab.

'''import fastparquet
train_df_sk.to_parquet('/content/gdrive/My Drive/models/train_df.parquet.gzip', compression='gzip')
val_df_sk.to_parquet('/content/gdrive/My Drive/models/val_df.parquet.gzip', compression='gzip')'''

In [0]:
# Read the parquet files as train and valid data frames.

import fastparquet
train_df = pd.read_parquet('/content/gdrive/My Drive/models/train_df.parquet.gzip')
val_df = pd.read_parquet('/content/gdrive/My Drive/models/val_df.parquet.gzip')

In [0]:
# Show rows form train dataframe

train_df.head()

In [0]:
# Show rows from valid dataframe

val_df.head()

### Train the model with resized 128x128 images

In [0]:
# Image Augmenters - Here image augmentations were not being applied as AutoAugment policy is being used subsequently
from albumentations import (Compose, HorizontalFlip, Cutout, PadIfNeeded, RandomCrop)
from albumentations import (Compose, HorizontalFlip, Cutout, PadIfNeeded, RandomCrop, CenterCrop, VerticalFlip)

def AUGMENTATIONS_INT_TRAIN(input_img, height = 224, width = 224):
    seq = Compose([#PadIfNeeded(height + 32,width + 32, p=1),
                   RandomCrop(height,width, p=1)
                   #HorizontalFlip(p=0.5),
                   #Cutout(num_holes=1, max_h_size=8, max_w_size=8, p=0.5)
                  ], p=1)
    output_img = seq(image = input_img)['image']
    return output_img

def AUGMENTATIONS_INT_TEST(input_img, height = 224, width = 224):
    seq = Compose([
                   RandomCrop(height,width, p=1)
                  ], p=1)
    output_img = seq(image = input_img)['image']
    return output_img


In [0]:
# Augmentation lambda functions for train and test

AUGMENTATIONS_TRAIN = lambda input_img: AUGMENTATIONS_INT_TRAIN(input_img, height = 128, width = 128)
AUGMENTATIONS_TEST = lambda input_img: AUGMENTATIONS_INT_TEST(input_img, height = 128, width = 128)


In [0]:
# create train and validation data generators. Here CIFRA10 AutoAugment policy is being applied for train generator.

new_shape = (128,128)

#policy = ImageNetPolicy()
policy = CIFAR10Policy()
#policy = SVHNPolicy()

train_gen = PersonDataGenerator(train_df, policy = policy, batch_size=512, new_shape = new_shape, augmentations = AUGMENTATIONS_TRAIN)
valid_gen = PersonDataGenerator(val_df, batch_size=512, shuffle=False, new_shape = new_shape, augmentations = AUGMENTATIONS_TEST)


In [0]:
# get number of output units from data
images, targets = next(iter(train_gen))
num_units = { k.split("_output")[0]:v.shape[1] for k, v in targets.items()}
num_units

In [0]:
# Show train images after applying AutoAugment policy
train_generator32 = PersonDataGenerator(train_df, policy = policy, batch_size=32, new_shape = new_shape, augmentations = AUGMENTATIONS_TRAIN)

x, y = next(iter(train_generator32))

plt.figure(figsize=(32, 16))
for i, (img, _) in enumerate(zip(x, y['bag_output'])):
    plt.subplot(4, 8, i+1)
    plt.axis('off')
    plt.imshow(img, interpolation="nearest")

In [0]:
# Show valid images , no augmentation or AutoAugment policy is applied for validation dataset.

valid_generator32 = PersonDataGenerator(val_df, batch_size=32, new_shape = new_shape, augmentations = AUGMENTATIONS_TRAIN)

x, y = next(iter(valid_generator32))

plt.figure(figsize=(32, 16))
for i, (img, _) in enumerate(zip(x, y['bag_output'])):
    plt.subplot(4, 8, i+1)
    plt.axis('off')
    plt.imshow(img, interpolation="nearest")

### Create ResNet18 model

In [0]:
# Classification models Zoo - Keras (and TensorFlow Keras) - https://github.com/qubvel/classification_models

!pip install git+https://github.com/qubvel/classification_models.git

In [0]:
# Obtain ResNet18 model without the classification head

from classification_models.tfkeras import Classifiers
ResNet18, preprocess_input = Classifiers.get('resnet18')
backbone = ResNet18((None, None, 3), weights=None, include_top=False)

In [0]:
# Add the classification head to ResNet18

'''backbone = VGG16(
    weights=None, 
    include_top=False, 
    input_tensor=Input(shape=(224, 224, 3))
)'''

neck = backbone.output
neck = GlobalAveragePooling2D()(neck)
#neck = Flatten(name="flatten")(neck)
neck = Dense(512, activation="relu")(neck)


def build_tower(in_layer):
    #neck = Dropout(0.2)(in_layer)
    #neck = Dense(256, activation="relu")(neck)
    neck = Dropout(0.2)(in_layer)
    neck = Dense(128, activation=None)(neck)
    neck = in_layer
    return neck


def build_head(name, in_layer):
    return Dense(
        num_units[name], activation="softmax", name=f"{name}_output"
    )(in_layer)

# heads
gender = build_head("gender", build_tower(neck))
image_quality = build_head("image_quality", build_tower(neck))
age = build_head("age", build_tower(neck))
weight = build_head("weight", build_tower(neck))
bag = build_head("bag", build_tower(neck))
footwear = build_head("footwear", build_tower(neck))
emotion = build_head("emotion", build_tower(neck))
pose = build_head("pose", build_tower(neck))


model = Model(
    inputs=backbone.input, 
    outputs=[gender, image_quality, age, weight, bag, footwear, pose, emotion]
)

In [0]:
# Define learning rate scheduler. Step learning rate is being used.

from tensorflow.keras.callbacks import LearningRateScheduler,ModelCheckpoint
def scheduler(epoch):
    if epoch < 20:
        return 0.1
    if 20 <= epoch < 40:
        return 0.02
    if 40 <= epoch < 60:
        return 0.004
    if epoch >= 60 :
        return 0.0008

# The below piece wise linear learning rate schedule defined is not used.
def lr_schedule(epoch):
  lr = 1.0/4.0
  scale_224 = 224 / 512
  scale_288 = 128 / 512
  lr = np.interp(epoch, [0, 5, 14, 16, 27, 32, 33, 35], [lr, lr*2,  lr * scale_224, lr / 10 * scale_224, lr / 100 * scale_224, lr / 100 * scale_288, lr / 1000 * scale_288, lr / 1000 * scale_288])
  print(lr)
  return lr

In [0]:
#loss_weights = {'gender_output': 1.0, 'image_quality_output': 0.0, 'age_output': 0.0, 'weight_output': 0.0, 'bag_output': 0.0, 'footwear_output': 0.0, 'pose_output': 0.0, 'emotion_output': 0.0}

# Define optimizer and compile the model


opt = SGD(lr=0.001, momentum=0.9, nesterov=True)
model.compile(
    optimizer=opt,
    loss="categorical_crossentropy", 
    #loss_weights=loss_weights, 
    metrics=["accuracy"]
)

In [0]:
# Display model summary

model.summary()

In [0]:
#class_weights = [{ 0 :1.28, 1 :1.0}, { 0 :1.0, 1 :3.31, 2: 1.97}, { 0 : 2.18 , 1 : 1.0, 2: 1.59, 3: 3.61, 4: 7.51 } ,{ 0 :1.0, 1 :9.78, 2 :2.72, 3 :10.27}, { 0 :1.66, 1 :5.8, 2: 1.0}, { 0 :1.19, 1 :2.38, 2: 1.0}, { 0 :3.77, 1 :1.0, 2: 2.79}, { 0 :6.56, 1 :6.14, 2: 1.0, 3: 12.25}]

In [0]:
# Train the model for 100 epochs

model.fit_generator(generator=train_gen,validation_data=valid_gen, use_multiprocessing=False,epochs=100, callbacks=[LearningRateScheduler(scheduler, verbose=1)], verbose=1)

# model.fit_generator(generator=train_gen,validation_data=valid_gen, use_multiprocessing=False,epochs=35, callbacks=[LearningRateScheduler(lr_schedule, verbose=1)], verbose=1)



In [0]:
model.save("/content/gdrive/My Drive/models/EIP4_Assign5_Size128_AutoAug_all_1.h5")

In [0]:
model.load_weights("/content/gdrive/My Drive/models/EIP4_Assign5_Size128_AutoAug_all_1.h5")

### Train the model with actual sized images i.e. 224x224 sized images

In [0]:
# Image Augmenters
from albumentations import (Compose, HorizontalFlip, Cutout, PadIfNeeded, RandomCrop)
from albumentations import (Compose, HorizontalFlip, Cutout, PadIfNeeded, RandomCrop, CenterCrop, VerticalFlip, Normalize, ShiftScaleRotate, RandomSizedCrop, RandomBrightnessContrast)

def AUGMENTATIONS_INT_TRAIN(input_img, height = 224, width = 224):
    seq = Compose([#PadIfNeeded(height + 32,width + 32, p=1),
                   #RandomSizedCrop((64,64),height,width, p=1),
                   #Normalize(),
                   #ShiftScaleRotate(),
                   #RandomBrightnessContrast(),
                   RandomCrop(height,width, p=1)
                   #HorizontalFlip(p=0.5),
                   #Cutout(num_holes=1, max_h_size=8, max_w_size=8, p=0.5)
                  ], p=1)
    output_img = seq(image = input_img)['image']
    return output_img

def AUGMENTATIONS_INT_TEST(input_img, height = 224, width = 224):
    seq = Compose([
                   RandomCrop(height,width, p=1)
                  ], p=1)
    output_img = seq(image = input_img)['image']
    return output_img

In [0]:
# Image augmentation lambda functions for train and validation

AUGMENTATIONS_TRAIN = lambda input_img: AUGMENTATIONS_INT_TRAIN(input_img, height = 224, width = 224)
AUGMENTATIONS_TEST = lambda input_img: AUGMENTATIONS_INT_TEST(input_img, height = 224, width = 224)

In [0]:
# create train and validation data generators. Use CIFAR10 AutoAugment policy.

new_shape = (224,224)

policy = CIFAR10Policy()

train_gen = PersonDataGenerator(train_df, policy = policy, batch_size=128, new_shape = new_shape, augmentations = AUGMENTATIONS_TRAIN)
valid_gen = PersonDataGenerator(val_df, batch_size=128, shuffle=False, new_shape = new_shape, augmentations = AUGMENTATIONS_TEST)

In [0]:
#loss_weights = {'gender_output': 5.0, 'image_quality_output': 20.0, 'age_output': 60.0, 'weight_output': 20.0, 'bag_output': 20.0, 'footwear_output': 20.0, 'pose_output': 20.0, 'emotion_output': 5.0}

# Checkpoint the best model i.e. model where validation loss improves.

from tensorflow.keras.callbacks import ModelCheckpoint
checkpointer = ModelCheckpoint("/content/gdrive/My Drive/models/EIP4_Assign5_Size224_all_Best_1.h5", monitor='val_loss', verbose=1, save_best_only=True)

model_callbacks=[checkpointer, LearningRateScheduler(scheduler, verbose=1)]

opt = SGD(lr=0.001, momentum=0.9, nesterov=True)
model.compile(
    optimizer=opt,
    loss="categorical_crossentropy", 
    #loss_weights=loss_weights, 
    metrics=["accuracy"]
)

In [0]:
# Train the model for 120 epochs

model.fit_generator(generator=train_gen,validation_data=valid_gen, use_multiprocessing=False,epochs=120, callbacks=model_callbacks, verbose=1)


In [0]:
model.save("/content/gdrive/My Drive/models/EIP4_Assign5_Size224_all_1.h5")

In [0]:
model.load_weights("/content/gdrive/My Drive/models/EIP4_Assign5_Size224_all_1.h5")

### Train the model for 200 epochs with CIFAR10 AutoAugment policy using 224x224 images

In [0]:
# Define learning rate scheduler. Step learning rate is being used.

from tensorflow.keras.callbacks import LearningRateScheduler,ModelCheckpoint
def scheduler(epoch):
    if epoch < 40:
        return 0.1
    if 40 <= epoch < 80:
        return 0.02
    if 80 <= epoch < 120:
        return 0.004
    if epoch >= 120 :
        return 0.0008

In [0]:
# Checkpoint and save the best model i.e. model with best validation loss

from tensorflow.keras.callbacks import ModelCheckpoint
checkpointer = ModelCheckpoint("/content/gdrive/My Drive/models/EIP4_Assign5_Size224_all_Best_2.h5", monitor='val_loss', verbose=1, save_best_only=True)

model_callbacks=[checkpointer, LearningRateScheduler(scheduler, verbose=1)]


In [0]:
# Train the model for 200 epochs

model.fit_generator(generator=train_gen,validation_data=valid_gen, use_multiprocessing=False,epochs=200, callbacks=model_callbacks, verbose=1)

In [0]:
model.save("/content/gdrive/My Drive/models/EIP4_Assign5_Size224_all_2.h5")

In [0]:
model.load_weights("/content/gdrive/My Drive/models/EIP4_Assign5_Size224_all_2.h5")

### Train the model for 200 epochs with RandomCrop, HorizontalFlip and Cutout(num_holes=20 and size 8x8) with 224x224 size images

In [0]:
# Image Augmenters - Not using any AutoAugment policies ,but use RandomCrop, HorizontalFlip and Cutout(num_holes=20 and size 8x8) augmentations.
from albumentations import (Compose, HorizontalFlip, Cutout, PadIfNeeded, RandomCrop)
from albumentations import (Compose, HorizontalFlip, Cutout, PadIfNeeded, RandomCrop, CenterCrop, VerticalFlip, Normalize, ShiftScaleRotate, RandomSizedCrop, RandomBrightnessContrast)

def AUGMENTATIONS_INT_TRAIN(input_img, height = 224, width = 224):
    seq = Compose([PadIfNeeded(height + 32,width + 32, p=1),
                   #RandomSizedCrop((64,64),height,width, p=1),
                   #Normalize(),
                   #ShiftScaleRotate(),
                   #RandomBrightnessContrast(),
                   RandomCrop(height,width, p=1),
                   HorizontalFlip(p=0.5),
                   Cutout(num_holes=20, max_h_size=8, max_w_size=8, p=0.5)
                  ], p=1)
    output_img = seq(image = input_img)['image']
    return output_img

def AUGMENTATIONS_INT_TEST(input_img, height = 224, width = 224):
    seq = Compose([
                   RandomCrop(height,width, p=1)
                  ], p=1)
    output_img = seq(image = input_img)['image']
    return output_img

In [0]:
# Augmentation lambda functions

AUGMENTATIONS_TRAIN = lambda input_img: AUGMENTATIONS_INT_TRAIN(input_img, height = 224, width = 224)
AUGMENTATIONS_TEST = lambda input_img: AUGMENTATIONS_INT_TEST(input_img, height = 224, width = 224)

In [0]:
# create train and validation data generators

new_shape = (224,224)

train_gen = PersonDataGenerator(train_df, policy = None, batch_size=128, new_shape = new_shape, augmentations = AUGMENTATIONS_TRAIN)
valid_gen = PersonDataGenerator(val_df, batch_size=128, shuffle=False, new_shape = new_shape, augmentations = AUGMENTATIONS_TEST)

In [0]:
# Define learning rate scheduler. Step learning rate is being used.

from tensorflow.keras.callbacks import LearningRateScheduler,ModelCheckpoint
def scheduler(epoch):
    if epoch < 40:
        return 0.1
    if 40 <= epoch < 80:
        return 0.02
    if 80 <= epoch < 120:
        return 0.004
    if epoch >= 120 :
        return 0.0008

In [0]:
# Checkpoint to save the best model i.e. model with best validation accuracy

from tensorflow.keras.callbacks import ModelCheckpoint
checkpointer = ModelCheckpoint("/content/gdrive/My Drive/models/EIP4_Assign5_Size224_all_Best_3.h5", monitor='val_loss', verbose=1, save_best_only=True)

model_callbacks=[checkpointer, LearningRateScheduler(scheduler, verbose=1)]



In [0]:
# Train the model for 200 epochs

model.fit_generator(generator=train_gen,validation_data=valid_gen, use_multiprocessing=False,epochs=200, callbacks=model_callbacks, verbose=1)


In [0]:
model.save("/content/gdrive/My Drive/models/EIP4_Assign5_Size224_all_3.h5")

In [0]:
model.load_weights("/content/gdrive/My Drive/models/EIP4_Assign5_Size224_all_3.h5")

### Train the model for 150 epochs with RandomCrop, HorizontalFlip and Cutout(num_holes=40 and size 8x8) with 224x224 size images

In [0]:
# Image Augmenters - No AutoAugment policies, but use RandomCrop, HorizontalFlip and Cutout(num_holes=40 and size 8x8) augmentations.
from albumentations import (Compose, HorizontalFlip, Cutout, PadIfNeeded, RandomCrop)
from albumentations import (Compose, HorizontalFlip, Cutout, PadIfNeeded, RandomCrop, CenterCrop, VerticalFlip, Normalize, ShiftScaleRotate, RandomSizedCrop, RandomBrightnessContrast)

def AUGMENTATIONS_INT_TRAIN(input_img, height = 224, width = 224):
    seq = Compose([PadIfNeeded(height + 32,width + 32, p=1),
                   #RandomSizedCrop((64,64),height,width, p=1),
                   #Normalize(),
                   #ShiftScaleRotate(),
                   #RandomBrightnessContrast(),
                   RandomCrop(height,width, p=1),
                   HorizontalFlip(p=0.5),
                   Cutout(num_holes=40, max_h_size=8, max_w_size=8, p=0.5)
                  ], p=1)
    output_img = seq(image = input_img)['image']
    return output_img

def AUGMENTATIONS_INT_TEST(input_img, height = 224, width = 224):
    seq = Compose([
                   RandomCrop(height,width, p=1)
                  ], p=1)
    output_img = seq(image = input_img)['image']
    return output_img

In [0]:
# Augmentation lambda functions

AUGMENTATIONS_TRAIN = lambda input_img: AUGMENTATIONS_INT_TRAIN(input_img, height = 224, width = 224)
AUGMENTATIONS_TEST = lambda input_img: AUGMENTATIONS_INT_TEST(input_img, height = 224, width = 224)

In [0]:
# create train and validation data generators

new_shape = (224,224)

train_gen = PersonDataGenerator(train_df, policy = None, batch_size=128, new_shape = new_shape, augmentations = AUGMENTATIONS_TRAIN)
valid_gen = PersonDataGenerator(val_df, batch_size=128, shuffle=False, new_shape = new_shape, augmentations = AUGMENTATIONS_TEST)

In [0]:
# Define learning rate scheduler. Step learning rate is being used.

from tensorflow.keras.callbacks import LearningRateScheduler,ModelCheckpoint
def scheduler(epoch):
    if epoch < 40:
        return 0.1
    if 40 <= epoch < 80:
        return 0.02
    if 80 <= epoch < 120:
        return 0.004
    if epoch >= 120 :
        return 0.0008

In [0]:
# Checkpoint the best model i.e. where validation loss is best/minimum

from tensorflow.keras.callbacks import ModelCheckpoint
checkpointer = ModelCheckpoint("/content/gdrive/My Drive/models/EIP4_Assign5_Size224_all_Best_4.h5", monitor='val_loss', verbose=1, save_best_only=True)

model_callbacks=[checkpointer, LearningRateScheduler(scheduler, verbose=1)]

In [0]:
model.fit_generator(generator=train_gen,validation_data=valid_gen, use_multiprocessing=False,epochs=150, callbacks=model_callbacks, verbose=1)


In [0]:
model.save("/content/gdrive/My Drive/models/EIP4_Assign5_Size224_all_4.h5")

### Load the best model and perform evaludation on the validation dataset

In [0]:
!pip install git+https://github.com/qubvel/classification_models.git

In [0]:
from classification_models.tfkeras import Classifiers
ResNet18, preprocess_input = Classifiers.get('resnet18')
backbone = ResNet18((None, None, 3), weights=None, include_top=False)

In [0]:


neck = backbone.output
neck = GlobalAveragePooling2D()(neck)
#neck = Flatten(name="flatten")(neck)
neck = Dense(512, activation="relu")(neck)


def build_tower(in_layer):
    #neck = Dropout(0.2)(in_layer)
    #neck = Dense(256, activation="relu")(neck)
    neck = Dropout(0.2)(in_layer)
    neck = Dense(128, activation=None)(neck)
    neck = in_layer
    return neck


def build_head(name, in_layer):
    return Dense(
        num_units[name], activation="softmax", name=f"{name}_output"
    )(in_layer)

# heads
gender = build_head("gender", build_tower(neck))
image_quality = build_head("image_quality", build_tower(neck))
age = build_head("age", build_tower(neck))
weight = build_head("weight", build_tower(neck))
bag = build_head("bag", build_tower(neck))
footwear = build_head("footwear", build_tower(neck))
emotion = build_head("emotion", build_tower(neck))
pose = build_head("pose", build_tower(neck))


best_model = Model(
    inputs=backbone.input, 
    outputs=[gender, image_quality, age, weight, bag, footwear, pose, emotion]
)

In [0]:
opt = SGD(lr=0.001, momentum=0.9, nesterov=True)
best_model.compile(
    optimizer=opt,
    loss="categorical_crossentropy", 
    #loss_weights=loss_weights, 
    metrics=["accuracy"]
)

In [0]:
# Load the model with best weights

best_model.load_weights("/content/gdrive/My Drive/models/EIP4_Assign5_Size224_all_Best_3.h5")

In [0]:
# Obtain results on validation dataset

val_results = best_model.evaluate(x=valid_gen, verbose=1)

In [0]:
# Print model metrics

print(best_model.metrics_names)

In [0]:
# Print validation results

print(val_results)

In [0]:
# Create the dictionary with metric names and metric values

result_dict = dict(zip(best_model.metrics_names, val_results))


In [0]:
# Print results dictionary

print(result_dict)

In [0]:
# Print validation accuracy results

import sys
sys.stdout.write("\033[0;0m")
for key,value in result_dict.items():
  if 'accuracy' in str(key):
    sys.stdout.write("\033[1;32m")
    sys.stdout.write("\033[;1m")
    print('{} : {}\n'.format(key,value))

## References & Attributions:



*   https://github.com/qubvel/classification_models.git
*   https://towardsdatascience.com/how-to-improve-your-image-classifier-with-googles-autoaugment-77643f0be0c9
*   https://github.com/DeepVoltaire/AutoAugment
*   https://arxiv.org/abs/1805.09501v1




*Disclaimer: The contents of this notebook are used for educational purposes i.e. for learning and research.*
